In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
import pyspark.sql 
from pyspark.sql import SQLContext
from pyspark.sql.functions import * 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
from pyspark.sql.types import Row


In [2]:
spark = SparkContext(master="local", appName="Proyecto")
sqlContext = SQLContext(spark)

In [3]:
!ls curso-apache-spark-platzi/proyecto/

ls: cannot access 'curso-apache-spark-platzi/proyecto/': No such file or directory


In [4]:
!ls /home/vagrant/curso-apache-spark-platzi/proyecto/

ls: cannot access '/home/vagrant/curso-apache-spark-platzi/proyecto/': No such file or directory


In [5]:
!head -n 5 /home/vagrant/curso-apache-spark-platzi/proyecto/athlete_events.csv

head: cannot open '/home/vagrant/curso-apache-spark-platzi/proyecto/athlete_events.csv' for reading: No such file or directory


In [6]:
def dropFirstRow(index,iterator):
     return iter(list(iterator)[1:]) #if index==0 else iterator

from pyspark.sql.functions import udf
colsInt = udf(lambda z: toInt(z), IntegerType())
colsFloat = udf(lambda z: toFloat(z), FloatType())

sqlContext.udf.register("colsInt", colsInt)
sqlContext.udf.register("colsFloat", colsFloat)

def toInt(s):
    if isinstance(s, str) == True:
        st = [str(ord(i)) for i in s]
        return(int(''.join(st)))
    else:
         return Null
    
def toFloat(s):
    if isinstance(s, str) == True:
        st = [str(ord(i)) for i in s]
        return(float(''.join(st)))
    else:
         return Null



In [7]:
path = "/home/spark/curso-apache-spark-platzi/proyecto/"

deportistaOlimpicoRDD =  spark.textFile(path+"deportista.csv").map(lambda line : line.split(","))
deportistaOlimpico2RDD = spark.textFile(path+"deportista2.csv").map(lambda line : line.split(","))
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpico2RDD)

deportistaOlimpicoRDD=deportistaOlimpicoRDD.mapPartitionsWithIndex(dropFirstRow)

deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l : (
int(l[0]),
l[1],
int(l[2]),
int(l[3]),
int(l[4]),
float(l[5]),
int(l[6])
))

schema = StructType([
StructField("deportista_id",IntegerType(),False)     ,
StructField("nombre",StringType(),False)        ,
StructField("genero",IntegerType(),False)        ,
StructField("edad",IntegerType(),True)      ,
StructField("altura",IntegerType(),True)        ,
StructField("peso",FloatType(),True)      ,
StructField("equipo_id",IntegerType(),True)     
])

deportistaOlimpicoDF = sqlContext.createDataFrame(deportistaOlimpicoRDD,schema)

In [8]:
paisesRDD = spark.textFile(path+"paises.csv",2).map(lambda line : line.split(","))
paisesRDD = paisesRDD.mapPartitionsWithIndex(dropFirstRow)

paisesRDD = paisesRDD.map(lambda l : (
int(l[0]),
l[1],
l[2]
))

schema = StructType([
StructField("id",IntegerType(),False),
StructField("equipo",StringType(),False),
StructField("sigla",StringType(),False)
])

paisesDF = sqlContext.createDataFrame(paisesRDD,schema)

In [9]:
deportesOlimpicosRDDSchema = StructType(
    [StructField("deporte_id",IntegerType(),False),
     StructField("deporte",StringType(),False)
    ])

deportesDF = sqlContext.read.schema(deportesOlimpicosRDDSchema).option("header","true").csv(path+"deporte.csv")

In [10]:
eventoSchema= StructType([
    StructField("evento_id",IntegerType(),False),
    StructField("nombre",StringType(),False),
    StructField("deporte_id",IntegerType(),False)
])

deportesOlimpicosDF = sqlContext.read.schema(eventoSchema).option("header","true").csv(path+"evento.csv")

In [11]:

juegoSchema = StructType([
    StructField("juego_id",IntegerType(),False),
    StructField("anio",StringType(),False),
    StructField("temporada",StringType(),False),
    StructField("ciudad",StringType(),False),
])
juegoDF = sqlContext.read.schema(juegoSchema).option("header","true").csv(path+"juegos.csv")

resultadoSchema = StructType([
    StructField("resultado_id",IntegerType(),False),
    StructField("medalla",StringType(),False),
    StructField("deporitsta_id",IntegerType(),False),
    StructField("juego_id",IntegerType(),False),
    StructField("evento_id",IntegerType(),False),
])
resultadoDF = sqlContext.read.schema(resultadoSchema).option("header","true").csv(path+"athlete_events.csv")

In [12]:
deportesDF.take(5)

[Row(deporte_id=1, deporte='Basketball'),
 Row(deporte_id=2, deporte='Judo'),
 Row(deporte_id=3, deporte='Football'),
 Row(deporte_id=4, deporte='Tug-Of-War'),
 Row(deporte_id=5, deporte='Speed Skating')]

In [13]:
deportesOlimpicosDF.take(5)

[Row(evento_id=1, nombre="Basketball Men's Basketball", deporte_id=1),
 Row(evento_id=2, nombre="Judo Men's Extra-Lightweight", deporte_id=2),
 Row(evento_id=3, nombre="Football Men's Football", deporte_id=3),
 Row(evento_id=4, nombre="Tug-Of-War Men's Tug-Of-War", deporte_id=4),
 Row(evento_id=5, nombre="Speed Skating Women's 500 metres", deporte_id=5)]

In [14]:
paisesDF.take(5)

[Row(id=1, equipo='30. Februar', sigla='AUT'),
 Row(id=2, equipo='A North American Team', sigla='MEX'),
 Row(id=3, equipo='Acipactli', sigla='MEX'),
 Row(id=4, equipo='Acturus', sigla='ARG'),
 Row(id=5, equipo='Afghanistan', sigla='AFG')]

In [15]:
juegoDF.take(5)

[Row(juego_id=1, anio='1896 Verano', temporada='1896', ciudad='Verano'),
 Row(juego_id=2, anio='1900 Verano', temporada='1900', ciudad='Verano'),
 Row(juego_id=3, anio='1904 Verano', temporada='1904', ciudad='Verano'),
 Row(juego_id=4, anio='1906 Verano', temporada='1906', ciudad='Verano'),
 Row(juego_id=5, anio='1908 Verano', temporada='1908', ciudad='Verano')]

In [16]:
deportistaOlimpicoDF.take(5)

[Row(deportista_id=1, nombre='A Dijiang', genero=1, edad=24, altura=180, peso=80.0, equipo_id=199),
 Row(deportista_id=2, nombre='A Lamusi', genero=1, edad=23, altura=170, peso=60.0, equipo_id=199),
 Row(deportista_id=3, nombre='Gunnar Nielsen Aaby', genero=1, edad=24, altura=0, peso=0.0, equipo_id=273),
 Row(deportista_id=4, nombre='Edgar Lindenau Aabye', genero=1, edad=34, altura=0, peso=0.0, equipo_id=278),
 Row(deportista_id=5, nombre='Christine Jacoba Aaftink', genero=2, edad=21, altura=185, peso=82.0, equipo_id=705)]

In [17]:
resultadoDF.take(5)

[Row(resultado_id=1, medalla='NA', deporitsta_id=1, juego_id=39, evento_id=1),
 Row(resultado_id=2, medalla='NA', deporitsta_id=2, juego_id=49, evento_id=2),
 Row(resultado_id=3, medalla='NA', deporitsta_id=3, juego_id=7, evento_id=3),
 Row(resultado_id=4, medalla='Gold', deporitsta_id=4, juego_id=2, evento_id=4),
 Row(resultado_id=5, medalla='NA', deporitsta_id=5, juego_id=36, evento_id=5)]

# Operaciones sobre DF

In [18]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: float (nullable = true)
 |-- equipo_id: integer (nullable = true)



In [19]:
colNombre = deportistaOlimpicoDF["nombre"]
colAltura = deportistaOlimpicoDF.altura
deportistaOlimpicoDF.select(colNombre).take(6)

[Row(nombre='A Dijiang'),
 Row(nombre='A Lamusi'),
 Row(nombre='Gunnar Nielsen Aaby'),
 Row(nombre='Edgar Lindenau Aabye'),
 Row(nombre='Christine Jacoba Aaftink'),
 Row(nombre='Per Knut Aaland')]

In [20]:
deportistaOlimpicoDF = deportistaOlimpicoDF.withColumnRenamed("genero","sexo").drop("altura")

In [21]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = true)
 |-- peso: float (nullable = true)
 |-- equipo_id: integer (nullable = true)



In [22]:
deportistaOlimpicoDF = deportistaOlimpicoDF.select("deportista_id","nombre","sexo",col("edad") \
                                                   .alias("edadAlJugar"),"equipo_id")

In [23]:
deportistaOlimpicoDF.take(10)

[Row(deportista_id=1, nombre='A Dijiang', sexo=1, edadAlJugar=24, equipo_id=199),
 Row(deportista_id=2, nombre='A Lamusi', sexo=1, edadAlJugar=23, equipo_id=199),
 Row(deportista_id=3, nombre='Gunnar Nielsen Aaby', sexo=1, edadAlJugar=24, equipo_id=273),
 Row(deportista_id=4, nombre='Edgar Lindenau Aabye', sexo=1, edadAlJugar=34, equipo_id=278),
 Row(deportista_id=5, nombre='Christine Jacoba Aaftink', sexo=2, edadAlJugar=21, equipo_id=705),
 Row(deportista_id=6, nombre='Per Knut Aaland', sexo=1, edadAlJugar=31, equipo_id=1096),
 Row(deportista_id=7, nombre='John Aalberg', sexo=1, edadAlJugar=31, equipo_id=1096),
 Row(deportista_id=8, nombre='Cornelia Cor Aalten Strannood ', sexo=2, edadAlJugar=18, equipo_id=705),
 Row(deportista_id=9, nombre='Antti Sami Aalto', sexo=1, edadAlJugar=26, equipo_id=350),
 Row(deportista_id=10, nombre='Einar Ferdinand Einari Aalto', sexo=1, edadAlJugar=26, equipo_id=350)]

In [24]:
jugadoresM=deportistaOlimpicoDF.filter((deportistaOlimpicoDF.sexo == 1)&(deportistaOlimpicoDF.edadAlJugar!= 0))

In [25]:
jugadoresM = jugadoresM.sort("edadAlJugar")
jugadoresM.show()

+-------------+--------------------+----+-----------+---------+
|deportista_id|              nombre|sexo|edadAlJugar|equipo_id|
+-------------+--------------------+----+-----------+---------+
|        71691|  Dimitrios Loundras|   1|         10|      333|
|        37333|Carlos Bienvenido...|   1|         11|      982|
|        49142|        Jan Hoffmann|   1|         12|      302|
|        40296|    Alain C. Giletti|   1|         12|      362|
|        24191| Philippe Cuelenaere|   1|         12|       96|
|       125092|tienne Nol Henri ...|   1|         12|      362|
|        42835|   Werner Grieshofer|   1|         12|       71|
|        48939|             Ho Gang|   1|         12|      738|
|        24054|  Joaquim Pedro Cruz|   1|         13|      688|
|        18068|Jrgen Wright Capp...|   1|         13|      742|
|        32832|        Gyrgy Erdlyi|   1|         13|      477|
|        55514|  Ian David Johnston|   1|         13|       66|
|        13520|   Csar Borja Pineda|   1

In [27]:
jugadoresF=deportistaOlimpicoDF.filter((deportistaOlimpicoDF.sexo == 2)&(deportistaOlimpicoDF.edadAlJugar!=0))

In [28]:
jugadoresF = jugadoresF.sort("edadAlJugar")
jugadoresF.show()

+-------------+--------------------+----+-----------+---------+
|deportista_id|              nombre|sexo|edadAlJugar|equipo_id|
+-------------+--------------------+----+-----------+---------+
|       126307|        Liana Vicens|   2|         11|      825|
|        47618|Sonja Henie Toppi...|   2|         11|      742|
|        22411|Magdalena Cecilia...|   2|         11|      413|
|        51268|      Beatrice Hutiu|   2|         11|      861|
|       118925|Megan Olwen Deven...|   2|         11|      413|
|        40129|    Luigina Giavotti|   2|         11|      507|
|        70616|          Liu Luyang|   2|         11|      199|
|        52070|        Etsuko Inada|   2|         11|      514|
|        76675|   Marcelle Matthews|   2|         11|      967|
|        43528|Antoinette Joyce ...|   2|         12|      172|
|        63816|Carolyn Patricia ...|   2|         12|      415|
|        25877|Olga Lucia de Ang...|   2|         12|      225|
|         5291|Marcia Arriaga La...|   2

In [29]:
jugadoresF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edadAlJugar: integer (nullable = true)
 |-- equipo_id: integer (nullable = true)



In [30]:
jugadoresM.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edadAlJugar: integer (nullable = true)
 |-- equipo_id: integer (nullable = true)



In [31]:
resultadoDF.printSchema()

root
 |-- resultado_id: integer (nullable = true)
 |-- medalla: string (nullable = true)
 |-- deporitsta_id: integer (nullable = true)
 |-- juego_id: integer (nullable = true)
 |-- evento_id: integer (nullable = true)



In [32]:
jugadoresF.join(resultadoDF, jugadoresF.deportista_id ==resultadoDF.deporitsta_id,"left") \
        .join(juegoDF, juegoDF.juego_id == resultadoDF.juego_id,"left") \
        .join(deportesOlimpicosDF, deportesOlimpicosDF.evento_id == resultadoDF.evento_id,"left") \
        .select(jugadoresF.nombre, col("edadAlJugar").alias("Edad al competir"),
                "medalla",col("anio").alias("Año de juego"),
                deportesOlimpicosDF.nombre.alias("Nombre disiplina")).show()

+--------------------+----------------+-------+-------------+--------------------+
|              nombre|Edad al competir|medalla| Año de juego|    Nombre disiplina|
+--------------------+----------------+-------+-------------+--------------------+
|Magdalena Cecilia...|              11| Silver|1936 Invierno|Figure Skating Wo...|
|Magdalena Cecilia...|              11|     NA|1932 Invierno|Figure Skating Wo...|
|    Luigina Giavotti|              11| Silver|  1928 Verano|Gymnastics Women'...|
|Sonja Henie Toppi...|              11|   Gold|1936 Invierno|Figure Skating Wo...|
|Sonja Henie Toppi...|              11|   Gold|1932 Invierno|Figure Skating Wo...|
|Sonja Henie Toppi...|              11|   Gold|1928 Invierno|Figure Skating Wo...|
|Sonja Henie Toppi...|              11|     NA|1924 Invierno|Figure Skating Wo...|
|      Beatrice Hutiu|              11|     NA|1968 Invierno|Figure Skating Wo...|
|        Etsuko Inada|              11|     NA|1936 Invierno|Figure Skating Wo...|
|   

In [33]:
jugadoresM.join(resultadoDF, jugadoresM.deportista_id ==resultadoDF.deporitsta_id,"left") \
        .join(juegoDF, juegoDF.juego_id == resultadoDF.juego_id,"left") \
        .join(deportesOlimpicosDF, deportesOlimpicosDF.evento_id == resultadoDF.evento_id,"left") \
        .select(jugadoresM.nombre, col("edadAlJugar").alias("Edad al competir"),
                "medalla",col("anio").alias("Año de juego"),
                deportesOlimpicosDF.nombre.alias("Nombre disiplina")).show()

+--------------------+----------------+-------+-------------+--------------------+
|              nombre|Edad al competir|medalla| Año de juego|    Nombre disiplina|
+--------------------+----------------+-------+-------------+--------------------+
|  Dimitrios Loundras|              10| Bronze|  1896 Verano|Gymnastics Men's ...|
|Carlos Bienvenido...|              11|     NA|  1992 Verano|Rowing Men's Coxe...|
| Philippe Cuelenaere|              12|     NA|  1984 Verano|Rowing Men's Coxe...|
|    Alain C. Giletti|              12|     NA|1960 Invierno|Figure Skating Me...|
|    Alain C. Giletti|              12|     NA|1956 Invierno|Figure Skating Me...|
|    Alain C. Giletti|              12|     NA|1952 Invierno|Figure Skating Me...|
|   Werner Grieshofer|              12|     NA|  1972 Verano|Rowing Men's Coxe...|
|             Ho Gang|              12|     NA|1988 Invierno|Figure Skating Me...|
|        Jan Hoffmann|              12| Silver|1980 Invierno|Figure Skating Me...|
|   

In [34]:
jugadoresTotal = jugadoresF.union(jugadoresM).distinct()
jugadoresTotal.count()

129202

In [35]:
jugadoresF.union(jugadoresM).count()

129202

Hacer dos dataframes nuevos
    1) poseerà el equipo de nacionalidad, año y ciudad del juego, medalla y disiplina y subdisiplina y nombre del atleta
 



In [38]:
medallistasXAnio = deportistaOlimpicoDF.join(resultadoDF, deportistaOlimpicoDF.deportista_id ==resultadoDF.deporitsta_id,"left") \
        .join(juegoDF, juegoDF.juego_id == resultadoDF.juego_id,"left") \
        .join(paisesDF,deportistaOlimpicoDF.equipo_id == paisesDF.id,"left") \
        .join(deportesOlimpicosDF, deportesOlimpicosDF.evento_id == resultadoDF.evento_id,"left") \
        .join(deportesDF,deportesOlimpicosDF.deporte_id == deportesDF.deporte_id,"left") \
        .select("sigla",
                "anio",
                "medalla",
                deportesOlimpicosDF.nombre.alias("Nombre subdisiplina"),
                deportesDF.deporte.alias("Nombre disiplina"),
                jugadoresM.nombre,   
                )


    2) poseerà solo medallas ganadoras (Oro,Plata y Bronze), pais, equipo

paisesDF.printSchema()

In [39]:
medallasXPais = resultadoDF.filter(resultadoDF.medalla != "NA") \
        .join(deportistaOlimpicoDF, deportistaOlimpicoDF.deportista_id == resultadoDF.deporitsta_id,"left") \
        .join(paisesDF, paisesDF.id == deportistaOlimpicoDF.equipo_id,"left") \
        .select("medalla","equipo","sigla").sort(col("sigla").desc())

In [40]:
medallistasXAnio.show()

+-----+-------------+-------+--------------------+----------------+--------------------+
|sigla|         anio|medalla| Nombre subdisiplina|Nombre disiplina|              nombre|
+-----+-------------+-------+--------------------+----------------+--------------------+
|  BRU|  2016 Verano|     NA|Athletics Women's...|       Athletics|Maizurah Abdul Rahim|
|  BRU|  2000 Verano|     NA|Shooting Men's Skeet|        Shooting|Jefri Kiko Bolkia...|
|  BRU|  1996 Verano|     NA|Shooting Men's Skeet|        Shooting|Jefri Kiko Bolkia...|
|  BRU|  2004 Verano|     NA|Athletics Men's 1...|       Athletics|     Jimmy Anak Ahar|
|  BRU|  2000 Verano|     NA|Athletics Men's 1...|       Athletics|         Haseri Asli|
|  BRU|  2016 Verano|     NA|Athletics Men's 1...|       Athletics|Mohamed Fakhri Is...|
|  BRU|  2012 Verano|     NA|Swimming Men's 20...|        Swimming|Anderson Chee Wei...|
|  BRU|  2012 Verano|     NA|Athletics Women's...|       Athletics|      Maziah Mahusin|
|  BRU|  2012 Verano|

In [44]:
medallistasXAnio.groupBy("nombre","anio","Nombre disiplina").count().show()

+--------------------+-------------+--------------------+-----+
|              nombre|         anio|    Nombre disiplina|count|
+--------------------+-------------+--------------------+-----+
|   Tofigh Jahanbakht|  1952 Verano|           Wrestling|    1|
|Jalal Ali Joe Kas...|  1968 Verano|           Athletics|    1|
|   Arash Miresmaeili|  2008 Verano|                Judo|    1|
|       Ahmed Yaghoti|  1976 Verano|          Water Polo|    1|
|Dmitry Yevgenyevi...|  1996 Verano|             Sailing|    1|
|        Yury Safonov|  1992 Verano|              Hockey|    1|
|Andrey Viktorovic...|1998 Invierno|          Ice Hockey|    1|
|      Chow Park Wing|  1960 Verano|             Sailing|    1|
|       Serkan Atasay|  2004 Verano|            Swimming|    1|
|Nataliya Volodymy...|  2004 Verano|           Athletics|    1|
|Oleksandr Mykhail...|  2008 Verano|              Boxing|    1|
|     Vitaliy Martsiv|2006 Invierno|Cross Country Skiing|    3|
|Liudmyla Ihorivna...|  1996 Verano|    

In [46]:
medallistasXAnio.select("Nombre subdisiplina").distinct().show(20)

+--------------------+
| Nombre subdisiplina|
+--------------------+
|Judo Men's Middle...|
|Swimming Men's 10...|
|Equestrianism Men...|
|Motorboating Mixe...|
|Shooting Men's Un...|
|Athletics Men's 1...|
|Gymnastics Women'...|
|Figure Skating Mi...|
|Cycling Men's Roa...|
|Cross Country Ski...|
|Tennis Mixed Doub...|
|Art Competitions ...|
|Gymnastics Men's ...|
|Weightlifting Wom...|
|Athletics Men's 3...|
|Cross Country Ski...|
|Sailing Mixed One...|
|Sailing Mixed 6 m...|
|Athletics Women's...|
|Swimming Women's ...|
+--------------------+
only showing top 20 rows



In [53]:
medallistasXAnio.where(col("Nombre subdisiplina").like("Judo Men's Middle%")) \
    .sort("anio") \
    .groupBy("Sigla","anio").count() \
    .show()

+-----+-----------+-----+
|Sigla|       anio|count|
+-----+-----------+-----+
|  PHI|1964 Verano|    2|
|  KOR|1964 Verano|    1|
|  FRA|1964 Verano|    2|
|  CRC|1964 Verano|    2|
|  VEN|1964 Verano|    1|
|  THA|1964 Verano|    1|
|  USA|1964 Verano|    1|
|  JPN|1964 Verano|    1|
|  ARG|1964 Verano|    1|
|  POR|1964 Verano|    1|
|  TPE|1964 Verano|    1|
|  GBR|1964 Verano|    1|
|  AUT|1964 Verano|    1|
|  VNM|1964 Verano|    2|
|  AUS|1964 Verano|    1|
|  GER|1964 Verano|    1|
|  NED|1964 Verano|    2|
|  BRA|1964 Verano|    1|
|  MEX|1964 Verano|    1|
| null|1964 Verano|    1|
+-----+-----------+-----+
only showing top 20 rows



In [80]:
temp1 = medallistasXAnio.filter(medallistasXAnio.medalla != "NA") \
    .sort("anio") \
    .groupBy("Sigla","anio","Nombre subdisiplina") \
    .count()


In [81]:
temp1.printSchema()

root
 |-- Sigla: string (nullable = true)
 |-- anio: string (nullable = true)
 |-- Nombre subdisiplina: string (nullable = true)
 |-- count: long (nullable = false)



In [82]:
temp1.show()

+-----+-----------+--------------------+-----+
|Sigla|       anio| Nombre subdisiplina|count|
+-----+-----------+--------------------+-----+
|  FRA|1896 Verano|Fencing Men's Foi...|    2|
|  FRA|1896 Verano|Cycling Men's 100...|    1|
|  FRA|1896 Verano|Cycling Men's 10,...|    2|
|  FRA|1896 Verano|Cycling Men's Sprint|    2|
|  FRA|1896 Verano|Athletics Men's 1...|    1|
|  FRA|1896 Verano|Cycling Men's 333...|    1|
|  FRA|1896 Verano|Fencing Men's Foi...|    1|
|  FRA|1896 Verano|Athletics Men's T...|    1|
|  SUI|1896 Verano|Gymnastics Men's ...|    1|
|  SUI|1896 Verano|Gymnastics Men's ...|    1|
|  SUI|1896 Verano|Gymnastics Men's ...|    1|
|  GRE|1896 Verano|Gymnastics Men's ...|    2|
|  GRE|1896 Verano|Swimming Men's 1,...|    1|
|  GRE|1896 Verano|Gymnastics Men's ...|    2|
|  GRE|1896 Verano|Tennis Men's Doubles|    2|
|  GRE|1896 Verano|Tennis Men's Singles|    2|
|  GRE|1896 Verano|Athletics Men's P...|    2|
|  GRE|1896 Verano|Shooting Men's Mi...|    1|
|  GRE|1896 V

In [83]:
temp1.groupBy("Sigla","anio"). \
    agg(sum("count").alias("Total de medallas"),\
        avg("count").alias("Medallas promedio")
       )\
    .show()

+-----+-----------+-----------------+------------------+
|Sigla|       anio|Total de medallas| Medallas promedio|
+-----+-----------+-----------------+------------------+
|  FRA|1896 Verano|               11|             1.375|
|  SUI|1896 Verano|                3|               1.0|
|  GRE|1896 Verano|               48|1.6551724137931034|
|  USA|1896 Verano|               20|1.6666666666666667|
|  DEN|1896 Verano|                6|               1.0|
|  HUN|1896 Verano|                6|               1.0|
|  GBR|1896 Verano|                9|             1.125|
|  AUT|1896 Verano|                5|               1.0|
|  AUS|1896 Verano|                3|               1.0|
|  GER|1896 Verano|               32|2.6666666666666665|
|  FRA|1900 Verano|              250| 4.310344827586207|
|  GER|1900 Verano|               45| 6.428571428571429|
|  SUI|1900 Verano|               21|               3.5|
|  LUX|1900 Verano|                1|               1.0|
|  NED|1900 Verano|            

2


In [84]:
medallasXPais.show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [85]:
medallasXPais.printSchema()

root
 |-- medalla: string (nullable = true)
 |-- equipo: string (nullable = true)
 |-- sigla: string (nullable = true)



In [ ]:
medallasXPais = resultadoDF.filter(resultadoDF.medalla != "NA") \
        .join(deportistaOlimpicoDF, deportistaOlimpicoDF.deportista_id == resultadoDF.deporitsta_id,"left") \
        .join(paisesDF, paisesDF.id == deportistaOlimpicoDF.equipo_id,"left") \
        .select("medalla","equipo","sigla").sort(col("sigla").desc())


Tablas usadas:
    resultado
    deportistaOlimpico
    paises

In [86]:
resultadoDF.registerTempTable("resultado")
deportistaOlimpicoDF.registerTempTable("deportista")
paisesDF.registerTempTable("paises")

In [93]:
sqlContext.sql("""SELECT medalla from  resultado""").take(5)

[Row(medalla='NA'),
 Row(medalla='NA'),
 Row(medalla='NA'),
 Row(medalla='Gold'),
 Row(medalla='NA')]

In [97]:
sqlContext.sql("""SELECT medalla,equipo,sigla FROM resultado r
                    JOIN deportista d
                    ON r.deporitsta_id = d.deportista_id
                    JOIN paises p
                    ON p.id = d.equipo_id
                    WHERE medalla <> "NA"
                    ORDER BY sigla DESC
                """).show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [99]:
resultadoDF.filter(resultadoDF.medalla != "NA") \
        .join(deportistaOlimpicoDF, deportistaOlimpicoDF.deportista_id == resultadoDF.deporitsta_id,"left") \
        .join(paisesDF, paisesDF.id == deportistaOlimpicoDF.equipo_id,"left") \
        .select("medalla","equipo","sigla").sort(col("sigla").desc()).show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [ ]:
#spark.stop()